# Getting Started With ASE

[The atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html) (ASE) is an extremely powerful tool for generating atomic structures and performing density functional theory calculations (DFT.) ASE allows you to manipulate atomic structures in a programitic way using the [ASE atoms object](https://wiki.fysik.dtu.dk/ase/ase/atoms.html). This is a `class` in python that can store the positions and identities of atoms in a structure and manipulate them in useful ways. This is pretty abstract, so here is an example of generating an atoms object:

In [1]:
!pip install ase
!pip install nglview
#!jupyter-nbextension enable nglview --py --sys-prefix  # this might be needed

In [2]:
from ase.atoms import Atoms # import the Atoms class from ASE

H2 = Atoms('HH', positions = [(0,0,0), (0,0,0.75)])
print(H2)

Atoms(symbols='H2', pbc=False)


The code above just generates an H$_2$ molecule, one hydrogen at the origin and one 0.75 angstroms up in the z direction. We can now manipulate it in interesting ways. Let's say we want to add a second H$_2$ molecule 2 angstroms away. We can do this by simply adding atoms objects to the one we already have

In [3]:
H2 = H2 + Atoms('H2', positions = [(2,0,0), (2,0,0.75)])
print(H2)
print(H2.positions)

Atoms(symbols='H4', pbc=False)
[[0.   0.   0.  ]
 [0.   0.   0.75]
 [2.   0.   0.  ]
 [2.   0.   0.75]]


So now we have have two H$_2$ molecules, we can shift all the atoms by simply adding an array to their positions

In [4]:
H2.positions = H2.positions + [0,0,1]
print(H2.positions)

[[0.   0.   1.  ]
 [0.   0.   1.75]
 [2.   0.   1.  ]
 [2.   0.   1.75]]


What does this look like though? We can visualize any structure using ASE's built in visualization tools

In [5]:
from ase.visualize import view

view(H2, viewer='ngl')

_ColormakerRegistry()

Atoms objects are also able to be indexed like lists. Each individual atom has an index and can be accessed in this way. When you call an index of an atoms object, you get an `Atom` object. This is just an object that represents a single atom. `Atoms` objects are simply a collection of `Atom` ojects

In [6]:
print(H2[0])
print(H2[0].position)
print(H2[0].symbol)

Atom('H', [0.0, 0.0, 1.0], index=0)
[0. 0. 1.]
H


Since we're doing periodic DFT, we want to put these atoms into a unit cell (a simulation box) to do this we want to use the `set_cell` method. Let's use a 10 angstrom box. We also want to write this to a file. ASE atoms objects have a `write` method that allows you to write to [almost any file type you can imagine](https://wiki.fysik.dtu.dk/ase/ase/io/io.html) (including .png images).

In [7]:
H2.set_cell([10,10,10])
H2.center() # this centers it in the unit cell
H2.write('2_hydrogens.xyz')

from ase.io import read # for reading files

H2_2 = read('2_hydrogens.xyz')

There are lots of useful things you can do to manipulate atoms, they are all documentated in the [ASE atoms object documentation](https://wiki.fysik.dtu.dk/ase/ase/atoms.html).

But having to have all the positions for the atoms in an atoms object is quite onerous. Luckily, there are [tools to build structures in ASE](https://wiki.fysik.dtu.dk/ase/ase/build/build.html). For example, there is a molecule function that holds the positions of lots of common molecules. Similarly, the `bulk` function contains tons of bulk structures for metals. We'll use this to generate surfaces a little later on

In [8]:
from ase.build import molecule, bulk

water = molecule('H2O')
print(water)
print(water.positions)

iron = bulk('Fe', cubic = True)
view(iron, viewer='ngl')

Atoms(symbols='OH2', pbc=False)
[[ 0.        0.        0.119262]
 [ 0.        0.763239 -0.477047]
 [ 0.       -0.763239 -0.477047]]


# ASE Calculators

ASE has the ability to attach a "calculator" to an atoms object. At a fundamental level, these are just classes that will return energies and forces. That means if can be a DFT program, a quantum chemistry program, or even just a classical leonard jones potential There are [a ton](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html) of calculators implmented. Here we will just use a simple one, the [EMT](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html) calculator.

The `EMT` calculator is just a simple pair potential calculator for a few metals. In practice it is a toy calculator used for testing.

You start by making an instance of your calculator, then using `set_calculator` to attach it to an atoms object. Once that is done you can call `get_potential_energy` and `get_forces` to calculate the energy and forces.

In [9]:
from ase.calculators.emt import EMT

calc = EMT()
water.set_calculator(calc)
energy = water.get_potential_energy()
forces = water.get_forces()

print(energy)
print(forces)

2.619811483369738
[[ 0.          0.          7.73836531]
 [ 0.          5.14779422 -3.86918266]
 [ 0.         -5.14779422 -3.86918266]]


In our group we use quatum espresso, VASP, and Abinit to do DFT in general.

A primary use of calculators is to perform structural optimizations. This allows us to find the "lowest energy configuration" of a given structure. ASE has [tools](https://wiki.fysik.dtu.dk/ase/ase/optimize.html) to do this. Below we're using the `BFGSLineSearch` optimization method

In [10]:
from ase.optimize import BFGSLineSearch

relax = BFGSLineSearch(atoms = water)
relax.run(fmax = 0.05) # relax the structure until the maximum force is 0.05 eV/A

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 17:00:58        2.619811*       7.7384
BFGSLineSearch:    1[  2] 17:00:58        1.879326*       0.0443


True

In [11]:
view(water, viewer='ngl')

# Calculating Adsorption Energies

Adsorption energies are the core of computational catalysis and surface science because they provide fundamental information about how a molecular intermediate interacts with a catalyst surface. Computational approaches are particularly valuable for adsorption energies because they are exceedingly difficult to measure experimentally. However, calculating adsorption energies also requires some effort. This section is meant to cover the basics of how to calculate an adsorption energy using DFT.

The adsorption energy is defined as the energy difference between the combined system and the separate systems:

$E_{ads} = E_{surf+ads} - E_{surf} - E_{gas}$

where $E_{surf+ads}$ is the combined system, $E_{surf}$ is the energy of the surface, and $E_{gas}$ is the energy of the molecule in the gas-phase. 

We'll start by making a CO molecule

In [12]:
from ase.build import molecule

gas = molecule('CO')

lets visualize it next.

In [13]:
from ase.visualize import view

view(gas, viewer='ngl')

To calculate the energy of this molecule, we're going to use the EMT calculator.

In [14]:
from ase.calculators.emt import EMT

gas.set_calculator(EMT())

Next we need to optimize the structure, because chances are it's not in its lowest energy configuration

In [15]:
from ase.optimize import QuasiNewton

dyn = QuasiNewton(gas)
dyn.run(fmax=0.05)


E_gas = gas.get_potential_energy()

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 17:00:58        0.786209*       2.3049
BFGSLineSearch:    1[  2] 17:00:58        0.672187*       0.3302
BFGSLineSearch:    2[  3] 17:00:58        0.670368*       0.0714
BFGSLineSearch:    3[  4] 17:00:58        0.670282*       0.0016


next let's build and optimize the slab. ASE has some nice tools for doing this.

In [16]:
from ase.build import fcc111, add_adsorbate

slab = fcc111('Pt', size = (2,2,4), vacuum = 10) # function for building 111 surfaces for fcc bulk struc


slab.set_calculator(EMT())
dyn = QuasiNewton(slab)
dyn.run(fmax=0.05)

E_slab = slab.get_potential_energy()

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 17:00:58        2.576181*       0.3811
BFGSLineSearch:    1[  1] 17:00:58        2.506582*       0.2684
BFGSLineSearch:    2[  2] 17:00:59        2.485247*       0.0649
BFGSLineSearch:    3[  3] 17:00:59        2.480666*       0.0440


In [17]:
view(slab, viewer='ngl')

Now we add the adsorpate and optimize That.

In [22]:
add_adsorbate(slab, gas, 2.3,'hcp')


slab.set_calculator(EMT())
dyn = QuasiNewton(slab)
dyn.run(fmax=0.05)

E_slab_ads = slab.get_potential_energy()
view(slab, viewer='ngl')

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 17:01:37       39.660019*      55.7187
BFGSLineSearch:    1[  1] 17:01:37       26.126396*      26.3357
BFGSLineSearch:    2[  2] 17:01:37       16.771833*      13.3545
BFGSLineSearch:    3[  3] 17:01:37       13.839315*      15.6944
BFGSLineSearch:    4[  4] 17:01:38       12.208646*      14.7715
BFGSLineSearch:    5[  6] 17:01:38        8.991629*       6.9573
BFGSLineSearch:    6[  7] 17:01:38        7.894238*       5.7533
BFGSLineSearch:    7[  9] 17:01:39        5.975976*       1.5938
BFGSLineSearch:    8[ 11] 17:01:39        4.999663*       1.5779
BFGSLineSearch:    9[ 13] 17:01:39        4.222764*       1.5933
BFGSLineSearch:   10[ 14] 17:01:39        4.027459*       1.5681
BFGSLineSearch:   11[ 16] 17:01:40        3.810620*       0.7191
BFGSLineSearch:   12[ 17] 17:01:40        3.748968*       0.4245
BFGSLineSearch:   13[ 18] 17:01:40       

In [19]:
view(slab, viewer='ngl')

Now, finally, we have the energy of the gas, the slab, and the slab with the adsorbate. We can now calculate the adsorption energy.

In [20]:
E_ads = E_slab_ads - E_slab - E_gas
print(E_ads)

-0.5331615226207451
